In [271]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line

import warnings
warnings.filterwarnings("ignore")

In [273]:
tweets_df = pd.read_csv('/Users/gazimahmud/Documents/Harvard/CSCI-171/finalproject/dataset-1111/covid19_tweets_final.csv', encoding='utf-8')
print("Number of tweets: ",len(tweets_df['full_text']))
tweets_df.head(50)

Number of tweets:  91448


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,full_text,...,date_short,weekday,hour,tweet_length,sentiment_tag_hf,sentiment_score_hf,sentiment_score_tb,subjectivity_score_tb,sentiment_score_nltk,id
0,Taya ❤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,Wildfires\nWar\nTaal Erruption\nJadine's break...,...,2020-01-27,0,0,112,NEGATIVE,0.997167,0.000000,0.000000,-0.7430,0
1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,Wuhan virus shot in the arm for health care st...,...,2020-01-27,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1
2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,.@bioRxiv: Transmission dynamics of 2019 novel...,...,2020-01-27,0,0,137,NEGATIVE,0.980284,0.000000,0.000000,0.5267,2
3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,"BREAKING: Coronavirus Hits 15% Fatality Rate, ...",...,2020-01-27,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3
4,Luke Waltham⁷,contactlukewaltham@gmail.com,22. Bisexual. Queer. He/They. Intersectional F...,2012-09-09 00:19:29,220029,9268,233087,True,2020-01-27 00:58:25,I find it disgustingly ironic that people are ...,...,2020-01-27,0,0,275,NEGATIVE,0.993612,0.000000,0.180000,-0.9524,4
5,CP24,Toronto,Official account of CP24 - Toronto and the GTA...,2009-01-28 04:56:13,1952779,371,1,True,2020-01-27 01:00:00,China temporarily bans wildlife trade in wake ...,...,2020-01-27,0,1,105,NEGATIVE,0.989554,0.000000,0.000000,0.0000,5
6,Amber 香港 😷✋☝️🇭🇰🆘🙏,Hong Kong,家庭煮婦，香港媽媽，支持自由、民主、人權、法治，願所有好人一生都平安🙏🤝🙇💪 I'm a H...,2019-09-04 20:28:23,2907,4914,59928,False,2020-01-27 01:15:57,#wuhan civilian said CCP promised them that by...,...,2020-01-27,0,1,304,NEGATIVE,0.996265,0.122500,0.565000,0.6597,6
7,Steven Lee Myers,NaN,The New York Times in Beijing. Formerly in Mos...,2009-04-21 14:14:37,15781,1366,2691,False,2020-01-27 01:21:52,“It’s precisely because Xi is so powerful that...,...,2020-01-27,0,1,159,NEGATIVE,0.954377,0.133333,0.800000,-0.5242,7
8,Solomon Yue,"Salem, Oregon, USA","Vice Chairman & CEO at Republicans Overseas, R...",2013-04-09 15:38:48,194950,424,61427,False,2020-01-27 01:31:59,#coronavirus is stolen from Canada by🇨🇳espiona...,...,2020-01-27,0,1,247,NEGATIVE,0.997793,-0.108333,0.175000,-0.9523,8
9,Peter Billerbeck,NaN,Current @HouseForeign | Fmr @TeamMoulton Sr Ad...,2010-08-09 22:32:27,1195,2165,4772,False,2020-01-27 01:42:56,"Given rapidly spreading #CoronaVirus, importan...",...,2020-01-27,0,1,278,NEGATIVE,0.998627,0.077679,0.553571,0.0000,9


In [274]:
tweets_df.columns

Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'full_text', 'hashtags', 'source', 'lang', 'source_top10',
       'date_short', 'weekday', 'hour', 'tweet_length', 'sentiment_tag_hf',
       'sentiment_score_hf', 'sentiment_score_tb', 'subjectivity_score_tb',
       'sentiment_score_nltk', 'id'],
      dtype='object')

In [275]:
tweets_df.head(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,full_text,...,date_short,weekday,hour,tweet_length,sentiment_tag_hf,sentiment_score_hf,sentiment_score_tb,subjectivity_score_tb,sentiment_score_nltk,id
0,Taya ❤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,Wildfires\nWar\nTaal Erruption\nJadine's break...,...,2020-01-27,0,0,112,NEGATIVE,0.997167,0.0,0.00,-0.7430,0
1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,Wuhan virus shot in the arm for health care st...,...,2020-01-27,0,0,102,NEGATIVE,0.999039,0.0,0.00,-0.0516,1
2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,.@bioRxiv: Transmission dynamics of 2019 novel...,...,2020-01-27,0,0,137,NEGATIVE,0.980284,0.0,0.00,0.5267,2
3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,"BREAKING: Coronavirus Hits 15% Fatality Rate, ...",...,2020-01-27,0,0,212,NEGATIVE,0.971413,0.0,0.45,-0.7430,3
4,Luke Waltham⁷,contactlukewaltham@gmail.com,22. Bisexual. Queer. He/They. Intersectional F...,2012-09-09 00:19:29,220029,9268,233087,True,2020-01-27 00:58:25,I find it disgustingly ironic that people are ...,...,2020-01-27,0,0,275,NEGATIVE,0.993612,0.0,0.18,-0.9524,4


In [276]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re 
nltk.download('words')


topics =[]
for index, row in tweets_df.iterrows():

    tweet = row['full_text']

    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake(language="english", min_length=2, max_length=4)

    # extracting the words by passing the text
    r.extract_keywords_from_text(tweet)
    b=r.get_ranked_phrases()
    clean_keyphrases = []
    for item in range(len(b)):            
        #if b[item] < 5 and b[item] > 0:
        phrase = b[item].replace("http","").replace("https","")
        clean_keyphrase = ' '.join([word for word in phrase.split() if word.isalpha()])
        n_gram = len(re.findall(r'\w+', clean_keyphrase))
        if (len(clean_keyphrase ) > 2 & n_gram > 0):
            clean_keyphrases.append(clean_keyphrase)
   
    #if (len(clean_keyphrases)>0):
    topics.append(clean_keyphrases)
    # assigning the key words to the new column for the corresponding movie
    row['topic_list'] = list(clean_keyphrases)
    
    
    c=r.get_ranked_phrases_with_scores()
    #print(c)
    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    #print(list(key_words_dict_scores.keys()))
    
    
tweets_df['topic_list'] = topics
    

[nltk_data] Downloading package words to
[nltk_data]     /Users/gazimahmud/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [277]:
tweets_df.columns

Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'full_text', 'hashtags', 'source', 'lang', 'source_top10',
       'date_short', 'weekday', 'hour', 'tweet_length', 'sentiment_tag_hf',
       'sentiment_score_hf', 'sentiment_score_tb', 'subjectivity_score_tb',
       'sentiment_score_nltk', 'id', 'topic_list'],
      dtype='object')

In [278]:
 tweets_df.shape

(91448, 25)

In [279]:
tweets_df.to_csv("/Users/gazimahmud/Documents/Harvard/CSCI-171/finalproject/dataset-1113/covid19_tweets_final.csv",encoding="utf-8")

In [280]:
tweets_df_verify = pd.read_csv('/Users/gazimahmud/Documents/Harvard/CSCI-171/finalproject/dataset-1113/covid19_tweets_final.csv', encoding='utf-8')
print("Number of tweets: ",len(tweets_df_verify['full_text']))
tweets_df_verify.head(50)

Number of tweets:  91448


,Unnamed: 0,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,weekday,hour,tweet_length,sentiment_tag_hf,sentiment_score_hf,sentiment_score_tb,subjectivity_score_tb,sentiment_score_nltk,id,topic_list
0,0,Taya ❤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,...,0,0,112,NEGATIVE,0.997167,0.000000,0.000000,-0.7430,0,"['year grabe january', 'coronavirus kobe']"
1,1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,"['casinos hurt s', 'wuhan virus shot', 'health..."
2,2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,...,0,0,137,NEGATIVE,0.980284,0.000000,0.000000,0.5267,2,"['aovlpgufvm s', 'transmission dynamics']"
3,3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,"['humanity s', 'coronavirus hits', 'infection ..."
4,4,Luke Waltham⁷,contactlukewaltham@gmail.com,22. Bisexual. Queer. He/They. Intersectional F...,2012-09-09 00:19:29,220029,9268,233087,True,2020-01-27 00:58:25,...,0,0,275,NEGATIVE,0.993612,0.000000,0.180000,-0.9524,4,"['fear getting', 'indigenous people', 'killed ..."
5,5,CP24,Toronto,Official account of CP24 - Toronto and the GTA...,2009-01-28 04:56:13,1952779,371,1,True,2020-01-27 01:00:00,...,0,1,105,NEGATIVE,0.989554,0.000000,0.000000,0.0000,5,['outbreak s']
6,6,Amber 香港 😷✋☝️🇭🇰🆘🙏,Hong Kong,家庭煮婦，香港媽媽，支持自由、民主、人權、法治，願所有好人一生都平安🙏🤝🙇💪 I'm a H...,2019-09-04 20:28:23,2907,4914,59928,False,2020-01-27 01:15:57,...,0,1,304,NEGATIVE,0.996265,0.122500,0.565000,0.6597,6,"['want better leader', 'good living standard',..."
7,7,Steven Lee Myers,NaN,The New York Times in Beijing. Formerly in Mos...,2009-04-21 14:14:37,15781,1366,2691,False,2020-01-27 01:21:52,...,0,1,159,NEGATIVE,0.954377,0.133333,0.800000,-0.5242,7,['late s']
8,8,Solomon Yue,"Salem, Oregon, USA","Vice Chairman & CEO at Republicans Overseas, R...",2013-04-09 15:38:48,194950,424,61427,False,2020-01-27 01:31:59,...,0,1,247,NEGATIVE,0.997793,-0.108333,0.175000,-0.9523,8,"['kill foreign enemies', 'becomes emperor shit..."
9,9,Peter Billerbeck,NaN,Current @HouseForeign | Fmr @TeamMoulton Sr Ad...,2010-08-09 22:32:27,1195,2165,4772,False,2020-01-27 01:42:56,...,0,1,278,NEGATIVE,0.998627,0.077679,0.553571,0.0000,9,"['senior admin official', 'note trump admin', ..."


In [281]:
frame = { 'topic': tweets_df["topic_list"].explode() } 
topics_df = pd.DataFrame(frame) 

In [282]:
topics_df.columns

Index(['topic'], dtype='object')

In [283]:
topics_df.head(10)


,topic
0,year grabe january
0,coronavirus kobe
1,casinos hurt s
1,wuhan virus shot
1,health care stocks
2,aovlpgufvm s
2,transmission dynamics
3,humanity s
3,coronavirus hits
3,infection rate


In [284]:
tweet_topic_df = tweets_df.join(topics_df)

In [300]:
tweet_topic_df.head(50)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,full_text,...,weekday,hour,tweet_length,sentiment_tag_hf,sentiment_score_hf,sentiment_score_tb,subjectivity_score_tb,sentiment_score_nltk,id,topic
0,Taya ❤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,Wildfires\nWar\nTaal Erruption\nJadine's break...,...,0,0,112,NEGATIVE,0.997167,0.000000,0.000000,-0.7430,0,year grabe january
0,Taya ❤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,Wildfires\nWar\nTaal Erruption\nJadine's break...,...,0,0,112,NEGATIVE,0.997167,0.000000,0.000000,-0.7430,0,coronavirus kobe
1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,Wuhan virus shot in the arm for health care st...,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,casinos hurt s
1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,Wuhan virus shot in the arm for health care st...,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,wuhan virus shot
1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,Wuhan virus shot in the arm for health care st...,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,health care stocks
2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,.@bioRxiv: Transmission dynamics of 2019 novel...,...,0,0,137,NEGATIVE,0.980284,0.000000,0.000000,0.5267,2,aovlpgufvm s
2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,.@bioRxiv: Transmission dynamics of 2019 novel...,...,0,0,137,NEGATIVE,0.980284,0.000000,0.000000,0.5267,2,transmission dynamics
3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,"BREAKING: Coronavirus Hits 15% Fatality Rate, ...",...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,humanity s
3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,"BREAKING: Coronavirus Hits 15% Fatality Rate, ...",...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,coronavirus hits
3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,"BREAKING: Coronavirus Hits 15% Fatality Rate, ...",...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,infection rate


In [285]:
tweet_topic_df.drop('topic_list', axis=1, inplace=True)

In [286]:
tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.90) 
               & (tweet_topic_df.sentiment_score_hf<=1) 
               & (tweet_topic_df.sentiment_tag_hf=='POSITIVE')]["topic"].value_counts()

social distancing          141
stay home                  124
global pandemic             85
stay safe                   79
coronavirus s               66
                          ... 
meaningful conversation      1
day moving averages          1
never drank                  1
zbtvagfkge s                 1
increased risk               1
Name: topic, Length: 39662, dtype: int64

In [287]:
positive90plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.90) 
               & (tweet_topic_df.sentiment_score_hf<=1) 
               & (tweet_topic_df.sentiment_tag_hf=='POSITIVE')]["topic"].value_counts()

In [288]:
positive90plus.head(50)

social distancing       141
stay home               124
global pandemic          85
stay safe                79
coronavirus s            66
corona virus             62
coronavirus pandemic     60
good news                59
tested positive          53
join us                  49
pandemic s               48
president trump          47
united states            42
new cases                42
loved ones               38
donald trump             37
take care                36
save lives               36
make sure                35
would like               34
mental health            32
please share             32
new york                 32
covid cases              30
looking forward          30
american people          28
well done                28
first time               27
wearing masks            27
god bless                27
look forward             27
coronavirus outbreak     27
get back                 26
joe biden                26
speedy recovery          26
great news          

In [289]:
positive80plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.80) 
               & (tweet_topic_df.sentiment_score_hf<=0.90) 
               & (tweet_topic_df.sentiment_tag_hf=='POSITIVE')]["topic"].value_counts()

In [290]:
positive80plus.head(20)

social distancing         26
stay home                 15
corona virus              10
coronavirus pandemic       9
donated million            8
tested positive            8
coronavirus s              8
stay safe                  8
public health              7
new cases                  6
help support               6
global pandemic            6
south korea                6
coronavirus outbreak s     6
hope bridge                6
first case                 5
president trump            5
covid cases                5
american people            5
coronavirus cases          5
Name: topic, dtype: int64

In [291]:
positive70plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.70) 
               & (tweet_topic_df.sentiment_score_hf<=0.80) 
               & (tweet_topic_df.sentiment_tag_hf=='POSITIVE')]["topic"].value_counts()
positive70plus.head(20)

social distancing         10
coronavirus pandemic       8
stay safe                  7
covid cases                5
coronavirus cases          5
united states              4
coronavirus s              4
new covid                  4
stay home                  4
breaking news              4
novel coronavirus          4
mental health              4
coronavirus outbreak s     4
pandemic s                 4
first time                 3
tested positive            3
bubonic plague             3
new zealand                3
looking forward            3
donated million            3
Name: topic, dtype: int64

In [292]:
positive60plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.60) 
               & (tweet_topic_df.sentiment_score_hf<=0.70) 
               & (tweet_topic_df.sentiment_tag_hf=='POSITIVE')]["topic"].value_counts()
positive60plus.head(20)

stay safe                 8
stay home                 6
corona virus              5
global pandemic           5
many people               5
coronavirus s             5
new cases                 4
pandemic s                4
total cases               4
tested positive           4
make sure                 4
social distancing         4
novel coronavirus         3
coronavirus outbreak s    3
coronavirus outbreak      3
take place                3
coronavirus pandemic      3
please take               3
new cases confirmed       3
united states             3
Name: topic, dtype: int64

In [293]:
positive50plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.50) 
               & (tweet_topic_df.sentiment_score_hf<=0.60) 
               & (tweet_topic_df.sentiment_tag_hf=='POSITIVE')]["topic"].value_counts()
positive50plus.head(20)

social distancing     8
tested positive       8
wear masks            5
stay home             5
corona virus          5
first time            4
social media          4
donald trump          4
south korea           4
american people       4
please share          3
global pandemic       3
last hours            3
united states         3
stay safe             3
coronavirus update    3
new covid             3
earn money            3
pandemic s            3
small businesses      3
Name: topic, dtype: int64

In [294]:
negative50plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.50) 
               & (tweet_topic_df.sentiment_score_hf<=0.60) 
               & (tweet_topic_df.sentiment_tag_hf=='NEGATIVE')]["topic"].value_counts()
negative50plus.head(20)

social distancing       12
tested positive          8
stay home                7
united states            7
coronavirus pandemic     7
social media             6
join cashgem             6
corona virus             6
earn money               6
covid cases              5
global pandemic          5
coronavirus cases        5
coronavirus crisis       4
small businesses         3
covid s                  3
mental health            3
coronavirus s            3
positive cases           3
wearing masks            3
human lives              3
Name: topic, dtype: int64

In [241]:
negative60plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.60) 
               & (tweet_topic_df.sentiment_score_hf<=0.70) 
               & (tweet_topic_df.sentiment_tag_hf=='NEGATIVE')]["topic"].value_counts()
negative60plus.head(20)

kind hearts asking help                 3
sydney talker tested positive           2
virus born economic crash               1
trump adviser steve cortes              1
overpriced martini isnt worth           1
govtimwalz hey communist dictator       1
national health commission said         1
reaching biggest indian film            1
multiple white house staff              1
nj golf club today                      1
per capita coronavirus infections       1
treasury secretary steven mnuchin       1
would count toward gdp                  1
new project apply blog                  1
unusually active litigation strategy    1
post covid onset glaucoma               1
next coronavirus aid bill               1
victoria posts deadliest day            1
dublin airport tomorrow morning         1
ivory coast read carefully              1
Name: topic, dtype: int64

In [242]:
negative70plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.70) 
               & (tweet_topic_df.sentiment_score_hf<=0.80) 
               & (tweet_topic_df.sentiment_tag_hf=='NEGATIVE')]["topic"].value_counts()
negative70plus.head(20)

investigate gates foundation garners          2
hard senator ernst squirms                    1
infectious disease experts cancel             1
dire public health example                    1
biggest daily figure anywhere                 1
coronavirus outbreak many army                1
chuck todd donald trump                       1
five best performing funds                    1
everyone pls stay calm                        1
help us better understand                     1
music festival last night                     1
moms soulfood cafe went                       1
discuss pandemic learning opportunities       1
church education system gaslighted            1
pickpocketbernie really believes communism    1
despite numerous challenges brought           1
young people catching coronavirus             1
ever done thus far                            1
using white house events                      1
facing unprecedented challenges due           1
Name: topic, dtype: int64

In [243]:
negative80plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.80) 
               & (tweet_topic_df.sentiment_score_hf<=0.90) 
               & (tweet_topic_df.sentiment_tag_hf=='NEGATIVE')]["topic"].value_counts()
negative80plus.head(20)

hsi said border states                   2
victorian premier daniel andrews         1
epidemic unbelievable neat bro           1
nsui national president shri             1
trump admin cancelled plans              1
neet jee amid pandemic                   1
cdc quietly revises yet                  1
new signals analytics report             1
word searches every week                 1
small businesses getting crushed         1
world health organisation said           1
consummate professional whose advice     1
media recycles old stories               1
predictive modeling without criticism    1
says captain america predicted           1
gop endlessly praises trump              1
still harassing innocents caught         1
home minister amit shah                  1
us every single year                     1
paralysed group joining packages         1
Name: topic, dtype: int64

In [296]:
negative90plus = tweet_topic_df[(tweet_topic_df.sentiment_score_hf >= 0.90) 
               & (tweet_topic_df.sentiment_score_hf<=1) 
               & (tweet_topic_df.sentiment_tag_hf=='NEGATIVE')]["topic"].value_counts()
negative90plus.head(20)

social distancing       744
coronavirus s           646
tested positive         472
stay home               382
corona virus            338
new cases               333
coronavirus pandemic    325
pandemic s              284
global pandemic         282
united states           278
coronavirus outbreak    220
coronavirus cases       215
donald trump            209
wear masks              194
covid deaths            176
white house             172
president trump         172
nursing homes           171
american people         165
go back                 162
Name: topic, dtype: int64

In [302]:
tweet_topic_df.to_csv("/Users/gazimahmud/Documents/Harvard/CSCI-171/finalproject/dataset-1113/covid19_tweets_final_denormalized.csv", encoding="utf-8")

In [301]:
tweet_topic_df.shape

(313766, 25)

In [305]:
tweets_topic_df = pd.read_csv('/Users/gazimahmud/Documents/Harvard/CSCI-171/finalproject/dataset-1113/covid19_tweets_final_denormalized.csv', encoding='utf-8')

In [307]:
tweets_topic_df.head(50)

,Unnamed: 0,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,weekday,hour,tweet_length,sentiment_tag_hf,sentiment_score_hf,sentiment_score_tb,subjectivity_score_tb,sentiment_score_nltk,id,topic
0,0,Taya ❤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,...,0,0,112,NEGATIVE,0.997167,0.000000,0.000000,-0.7430,0,year grabe january
1,0,Taya ❤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,...,0,0,112,NEGATIVE,0.997167,0.000000,0.000000,-0.7430,0,coronavirus kobe
2,1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,casinos hurt s
3,1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,wuhan virus shot
4,1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,health care stocks
5,2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,...,0,0,137,NEGATIVE,0.980284,0.000000,0.000000,0.5267,2,aovlpgufvm s
6,2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,...,0,0,137,NEGATIVE,0.980284,0.000000,0.000000,0.5267,2,transmission dynamics
7,3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,humanity s
8,3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,coronavirus hits
9,3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,infection rate


In [148]:
tweets_topic_df.shape

(91448, 25)

In [78]:
tweets_topic_df['sentiment_score_hf'].describe()

count    91448.000000
mean         0.964215
std          0.082669
min          0.500012
25%          0.979205
50%          0.993842
75%          0.998008
max          0.999882
Name: sentiment_score_hf, dtype: float64

In [ ]:
tweets_topic_df.describe()

In [ ]:
99-100, 95-99, 90-85, 85-90, 80-85, 75-80. 70-75, 65-70, 0-65

In [ ]:
positive = tweets_topic_df[tweets_topic_df["sentiment"]=="POSITIVE"]

In [ ]:
positive.describe()

In [ ]:
positive.shape


In [ ]:
negative = tweets_topic_df[tweets_topic_df["sentiment"]=="POSITIVE"]

In [ ]:
negative.shape

In [85]:
tweets_topic_df = pd.read_csv('/Users/gazimahmud/Downloads/covid19_tweets_final.csv', encoding='latin1')

In [86]:
tweets_topic_df.head(10)

,Unnamed: 0,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,weekday,hour,tweet_length,sentiment_tag_hf,sentiment_score_hf,sentiment_score_tb,subjectivity_score_tb,sentiment_score_nltk,id,topic_list
0,0,Taya Ã¢ÂÂ¤,NaN,I don't follow back,2011-10-01 05:55:50,283,0,8324,False,2020-01-27 00:24:01,...,0,0,112,NEGATIVE,0.997167,0.000000,0.000000,-0.7430,0,"['death january feels like', 'year grabe janua..."
1,1,Eric Ng,Hong Kong,"Hong Kong-based senior business reporter, Sout...",2015-02-23 12:28:27,1000,191,1,True,2020-01-27 00:37:26,...,0,0,102,NEGATIVE,0.999039,0.000000,0.000000,-0.0516,1,"['casinos hurt s', 'wuhan virus shot', 'health..."
2,2,Greg Folkers,"Washington, D.C., U.S.A.","Washington, DC",2009-05-21 17:03:17,4188,178,875,False,2020-01-27 00:50:36,...,0,0,137,NEGATIVE,0.980284,0.000000,0.000000,0.5267,2,"['s', 'aovlpgufvm s', 'transmission dynamics',..."
3,3,Deplorable Chump Texanon,"Texas, USA","God, Family, Country. God Wins! I love my won...",2009-03-26 20:31:39,675,987,3491,False,2020-01-27 00:58:18,...,0,0,212,NEGATIVE,0.971413,0.000000,0.450000,-0.7430,3,"['lancet publishes early study', 's', 'coronav..."
4,4,Luke WalthamÃ¢ÂÂ·,contactlukewaltham@gmail.com,22. Bisexual. Queer. He/They. Intersectional F...,2012-09-09 00:19:29,220029,9268,233087,True,2020-01-27 00:58:25,...,0,0,275,NEGATIVE,0.993612,0.000000,0.180000,-0.9524,4,"['Ã¢ fear getting', 'racist towards asian peop..."
5,5,CP24,Toronto,Official account of CP24 - Toronto and the GTA...,2009-01-28 04:56:13,1952779,371,1,True,2020-01-27 01:00:00,...,0,1,105,NEGATIVE,0.989554,0.000000,0.000000,0.0000,5,"['outbreak s', 's']"
6,6,Amber Ã©Â¦ÂÃ¦Â¸Â¯ Ã°ÂÂÂ·Ã¢ÂÂÃ¢ÂÂÃ¯Â¸ÂÃ...,Hong Kong,Ã¥Â®Â¶Ã¥ÂºÂ­Ã§Â Â®Ã¥Â©Â¦Ã¯Â¼ÂÃ©Â¦ÂÃ¦Â¸Â¯Ã¥Âª...,2019-09-04 20:28:23,2907,4914,59928,False,2020-01-27 01:15:57,...,0,1,304,NEGATIVE,0.996265,0.122500,0.565000,0.6597,6,"['want better leader', 'good living standard',..."
7,7,Steven Lee Myers,NaN,The New York Times in Beijing. Formerly in Mos...,2009-04-21 14:14:37,15781,1366,2691,False,2020-01-27 01:21:52,...,0,1,159,NEGATIVE,0.954377,0.133333,0.800000,-0.5242,7,"['Ã¢ s', 'Ã¢ itÃ¢']"
8,8,Solomon Yue,"Salem, Oregon, USA","Vice Chairman & CEO at Republicans Overseas, R...",2013-04-09 15:38:48,194950,424,61427,False,2020-01-27 01:31:59,...,0,1,247,NEGATIVE,0.997793,-0.108333,0.175000,-0.9523,8,"['kill foreign enemies', 'becomes emperor shit..."
9,9,Peter Billerbeck,NaN,Current @HouseForeign | Fmr @TeamMoulton Sr Ad...,2010-08-09 22:32:27,1195,2165,4772,False,2020-01-27 01:42:56,...,0,1,278,NEGATIVE,0.998627,0.077679,0.553571,0.0000,9,"['single senior us official', 'senior admin of..."


In [87]:
tweets_topic_df.groupby('topic_list').id.apply(lambda x: x.unique().tolist())

topic_list
['', '', '', 'samples collected', '', 'active cases', 'total cases', 'recovered cases', 'covid', 'people']                                              [58037]
['', '', 'usa covid', 'june', 'pm', 'stats', 'pm']                                                                                                      [32332]
['', '']                                                                                                                                                [87958]
['', 'canÃ¢']                                                                                                                                           [20022]
['', 'cnn maybe']                                                                                                                                       [55631]
                                                                                                                                    ...                        
['Ã°', 'Ã°', 'young doctor', 

In [107]:
tweets_topic_df.dtypes

Unnamed: 0                  int64
user_name                  object
user_location              object
user_description           object
user_created               object
user_followers              int64
user_friends                int64
user_favourites             int64
user_verified                bool
date                       object
full_text                  object
hashtags                   object
source                     object
lang                       object
source_top10               object
date_short                 object
weekday                     int64
hour                        int64
tweet_length                int64
sentiment_tag_hf           object
sentiment_score_hf        float64
sentiment_score_tb        float64
subjectivity_score_tb     float64
sentiment_score_nltk      float64
id                          int64
topic_list               category
dtype: object

In [104]:
tweets_topic_df["topic_list"] = tweets_topic_df["topic_list"].astype('category')

In [108]:
s = tweets_topic_df.apply(lambda x: pd.Series(x['topic_list']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'topic'

In [109]:
tweets_topic_df["topic_list"].explode()

0        ['death january feels like', 'year grabe janua...
1        ['casinos hurt s', 'wuhan virus shot', 'health...
2        ['s', 'aovlpgufvm s', 'transmission dynamics',...
3        ['lancet publishes early study', 's', 'coronav...
4        ['Ã¢ fear getting', 'racist towards asian peop...
                               ...                        
91443    ['next year s', 'moderna ceo says', 'coronavir...
91444                                            ['fyi s']
91445    ['feel like covid', 'feel better soon', 'world...
91446          ['donate s', 'raising money', 'bible says']
91447        ['eye opening tbh', 'okay physically', 'iÃ¢']
Name: topic_list, Length: 91448, dtype: category
Categories (85683, object): ['['', '', '', 'samples collected', '', 'active..., '['', '', 'usa covid', 'june', 'pm', 'stats', ..., '['', '']', '['', 'canÃ¢']', ..., '['Ã°', 'Ã°', 'Ã°', 'us forever', 'forever', '..., '['Ã°', 'Ã°', 'Ã°', 'Ã°', 'Ã°', 's', 'Ã°', 's'..., '['Ã°']', '[]']

In [110]:
keys = ['panda1', 'panda2', 'panda3']
values = [['eats', 'shoots'], ['shoots', 'leaves'], ['eats', 'leaves']]
temp = pd.DataFrame({'keys': keys, 'values': values})
temp.head(10)

,keys,values
0,panda1,"[eats, shoots]"
1,panda2,"[shoots, leaves]"
2,panda3,"[eats, leaves]"


In [111]:
temp['values'].explode()

0      eats
0    shoots
1    shoots
1    leaves
2      eats
2    leaves
Name: values, dtype: object

In [112]:
temp.dtypes

keys      object
values    object
dtype: object